In [2]:
import datetime as dt
from utility import (
    fd_alive_funds,
    fd_basicinfo,
    fd_typeclass,
    nearest_report_date
)
import pandas as pd

In [3]:
# 获取回溯的报告日

today = dt.datetime.today().strftime("%Y%m%d")

In [4]:
# 获取所有目标QDII基金（清算结束日期晚于当前日）

security_ids = fd_alive_funds(today, 601)

In [5]:
basic_info = fd_basicinfo(security_ids)
type_info = fd_typeclass(security_ids)

In [6]:
df = pd.merge(type_info, basic_info, on="SECURITYID")

In [7]:
df.L3NAME.unique()

array(['QDII股票型基金', 'QDII混合基金', 'QDII债券基金', 'QDII指数股票型基金', 'QDII商品基金',
       'QDII房地产信托基金', '灵活配置型基金(股票上限95%)（A类）'], dtype=object)

In [8]:
# 分类
df.loc[(df["L3NAME"].isin(["QDII股票型基金", "QDII全球股票型基金", "QDII指数股票型基金"])), "QDII型（子类）"] = "QDII股票型基金"
df.loc[df["L3NAME"] == "QDII债券基金", "QDII型（子类）"] = "QDII债券型基金"
df.loc[(df["L3NAME"].isin(["QDII商品基金", "QDII房地产信托基金"])), "QDII型（子类）"] = "QDII另类投资基金"
df.loc[(df["L3NAME"].isin(["QDII混合基金", "QDII分级子基金"])), "QDII型（子类）"] = "QDII混合型基金"

In [9]:
df.groupby("QDII型（子类）").count()

,SECURITYID,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE
QDII型（子类）,,,,,,,,,,,,
QDII债券型基金,38,38,38,38,38,38,38,38,37,38,38,38
QDII另类投资基金,10,10,10,10,10,10,10,10,6,10,10,10
QDII混合型基金,27,27,27,27,27,27,27,27,10,27,27,27
QDII股票型基金,53,53,53,53,53,53,53,53,40,53,53,53
